In [6]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [8]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [9]:
restrict_48_hours = True
include_notes = True
include_cxr = True
include_ecg = True
standard_scale = True
include_missing = True

In [10]:
ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_new.pkl"))
ireg_vitals_ts_df = ireg_vitals_ts_df.drop(columns=["hosp_time_delta"])
ireg_vitals_ts_df.rename(columns={'icu_time_delta': 'timedelta'}, inplace=True)
imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_new.pkl"))

# ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))
# imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_icu.pkl"))
print(ireg_vitals_ts_df.columns)
print(imputed_vitals.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red Blood Cells', 'Respiratory Rate', 'Sodium', 'Systolic BP',
       'Urea Nitrogen', 'Vancomycin', 'White Blood Cells'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red

In [11]:
print(len(ireg_vitals_ts_df.columns))

34


In [12]:

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] >= 0]
imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] >= 0]

if restrict_48_hours:
    ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] <= 48]
    imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] <= 48]

In [13]:
if include_notes:
    notes_df = pd.read_pickle(os.path.join(output_dir, "DuoAllchunk_icu_notes_text_embeddings.pkl"))
    print(notes_df.columns)
    # notes_df = pd.read_pickle(os.path.join(output_dir, "notes_text.pkl"))
    notes_df = notes_df[notes_df['stay_id'].notnull()]

    notes_df = notes_df[notes_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        notes_df = notes_df[notes_df['icu_time_delta'] <= 48]

if include_cxr:
    cxr_df = pd.read_pickle(os.path.join(output_dir, "cxr_embeddings_stay.pkl"))
    print(cxr_df.columns)
    cxr_df = cxr_df[cxr_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        cxr_df = cxr_df[cxr_df['icu_time_delta'] <= 48]

if include_ecg:
    ecg_df = pd.read_pickle(os.path.join(output_dir, "ecg_embeddings_icu.pkl"))
    print(ecg_df.columns)
    ecg_df = ecg_df[ecg_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        ecg_df = ecg_df[ecg_df['icu_time_delta'] <= 48]

Index(['note_id_x', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'stay_id', 'icu_time_delta',
       'hosp_time_delta', 'note_id_y', 'dis_text', 'biobert_embeddings',
       'longformer_embeddings'],
      dtype='object')
Index(['dicom_id', 'subject_id', 'study_id',
       'PerformedProcedureStepDescription', 'ViewPosition',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'densefeatures',
       'predictions', 'cxrtime', 'hadm_id', 'stay_id', 'icu_time_delta',
       'hosp_time_delta'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'icu_time_delta', 'ecg_time',
       'path', 'embeddings'],
      dtype='object')


In [14]:
echo_notes = pd.read_csv("/data/wang/junh/datasets/multimodal/haim_replication/echo_embed.csv")
ecg_notes = pd.read_csv("/data/wang/junh/datasets/multimodal/haim_replication/ecg_embed.csv")

In [15]:
keys = pd.read_csv("/data/wang/junh/datasets/physionet.org/files/haim-multimodal/1.0.1/haim_mimiciv_key_ids.csv")

In [16]:
echo_notes.drop(columns=["Unnamed: 0"], inplace=True)
echo_notes = pd.concat([keys, echo_notes], axis=1)
ecg_notes.drop(columns=["Unnamed: 0"], inplace=True)
ecg_notes = pd.concat([keys, ecg_notes], axis=1)

In [17]:
notes_df = notes_df.merge(echo_notes, on=['subject_id', 'stay_id', 'hadm_id'], how="left")
notes_df = notes_df.merge(ecg_notes, on=['subject_id', 'stay_id', 'hadm_id'], how="left")

/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():
/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [18]:
print("Number of stays in ecg: ", ecg_df['stay_id'].nunique())
print("Number of stays in cxr: ", cxr_df['stay_id'].nunique())
print("Number of stays in notes: ", notes_df['stay_id'].nunique())

Number of stays in ecg:  32937
Number of stays in cxr:  13761
Number of stays in notes:  55050


In [19]:
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

if restrict_48_hours:
    icustays_df = icustays_df[icustays_df['los'] >= 2]

In [20]:
valid_stay_ids = icustays_df['stay_id'].unique()

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(valid_stay_ids)]
imputed_vitals = imputed_vitals[imputed_vitals['stay_id'].isin(valid_stay_ids)]

if include_notes:
    notes_df = notes_df[notes_df['stay_id'].isin(valid_stay_ids)]

if include_cxr:
    cxr_df = cxr_df[cxr_df['stay_id'].isin(valid_stay_ids)]

if include_ecg:
    ecg_df = ecg_df[ecg_df['stay_id'].isin(valid_stay_ids)]

In [21]:
print("Number of stays in ecg: ", ecg_df['stay_id'].nunique())
print("Number of stays in cxr: ", cxr_df['stay_id'].nunique())
print("Number of stays in notes: ", notes_df['stay_id'].nunique())

Number of stays in ecg:  18272
Number of stays in cxr:  8781
Number of stays in notes:  31721


In [22]:
print(notes_df.columns)

Index(['note_id_x', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'stay_id', 'icu_time_delta',
       'hosp_time_delta', 'note_id_y', 'dis_text', 'biobert_embeddings',
       'longformer_embeddings', 'echo_embed', 'ecg_embed'],
      dtype='object')


In [23]:
print(ireg_vitals_ts_df.shape)

(3014705, 34)


In [24]:
admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"))
admissions_df = admissions_df.rename(columns={"hospital_expire_flag": "died"})
admissions_df = admissions_df[["subject_id", "hadm_id", "died"]]

In [25]:

if not include_missing:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        unique_stays = np.intersect1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with notes: {len(unique_stays)}")

    if include_cxr:
        unique_stays = np.intersect1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with cxr: {len(unique_stays)}")

    if include_ecg:
        unique_stays = np.intersect1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with ecg: {len(unique_stays)}")
else:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        # Get stays with either TS or notes
        unique_stays = np.union1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with either TS or notes: {len(unique_stays)}")
    
    if include_cxr:
        unique_stays = np.union1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr: {len(unique_stays)}")
    
    if include_ecg:
        unique_stays = np.union1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr, ecg: {len(unique_stays)}")

Number of stays with vitals: 35131
Number of stays with either TS or notes: 35131
Number of stays with either TS, notes, cxr: 35131
Number of stays with either TS, notes, cxr, ecg: 35131


In [26]:
# Create train, val, test splits
np.random.seed(0)
np.random.shuffle(unique_stays)
train_stays = unique_stays[:int(0.7*len(unique_stays))]
val_stays = unique_stays[int(0.7*len(unique_stays)):int(0.85*len(unique_stays))]
test_stays = unique_stays[int(0.85*len(unique_stays)):]

In [27]:
train_ireg_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(train_stays)].copy()
train_imputed_df = imputed_vitals[imputed_vitals['stay_id'].isin(train_stays)].copy()

cols = train_ireg_ts_df.columns.tolist()
cols = [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'timedelta']]

if standard_scale:
    for col in cols:
        scaler = StandardScaler()
        scaler.fit(train_ireg_ts_df[[col]])
        ireg_vitals_ts_df[col] = scaler.transform(ireg_vitals_ts_df[[col]])

        scaler = StandardScaler()
        scaler.fit(train_imputed_df[[col]])
        imputed_vitals[col] = scaler.transform(imputed_vitals[[col]])


In [28]:
print(train_ireg_ts_df.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red Blood Cells', 'Respiratory Rate', 'Sodium', 'Systolic BP',
       'Urea Nitrogen', 'Vancomycin', 'White Blood Cells'],
      dtype='object')


In [ ]:
def get_stay_list(stays):
    stays_list = []

    for curr_stay in tqdm(stays):
        
        curr_stay_ireg = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()
        #print(f"Initial irregular time series shape (before dropping columns): {curr_stay_ireg.shape}")
        curr_stay_imputed = imputed_vitals[imputed_vitals['stay_id'] == curr_stay].copy()

        if len(curr_stay_ireg) == 0:
            continue

        curr_stay_dict = {}
        curr_stay_dict['name'] = curr_stay_ireg['subject_id'].iloc[0]
        curr_stay_dict['hadm_id'] = curr_stay_ireg['hadm_id'].iloc[0]
        curr_stay_dict['stay_id'] = curr_stay
        curr_stay_dict['ts_tt'] = curr_stay_ireg['timedelta'].values

        curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        ireg_ts_mask = curr_stay_ireg.notnull()
        curr_stay_ireg.fillna(0, inplace=True)
        curr_stay_dict['irg_ts'] = curr_stay_ireg.values
        curr_stay_dict['irg_ts_mask'] = ireg_ts_mask.values.astype(int)

        curr_stay_imputed.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        curr_stay_dict['reg_ts'] = curr_stay_imputed.values

        if include_notes:
            curr_stay_notes = notes_df[notes_df['stay_id'] == curr_stay].copy()

            if len(curr_stay_notes) == 0:
                curr_stay_dict['text_data'] = []
                curr_stay_dict['text_time_to_end'] = []
                curr_stay_dict['text_embeddings'] = []
                curr_stay_dict['text_missing'] = 1
            else:
                curr_stay_dict['text_data'] = curr_stay_notes['text'].tolist() + curr_stay_notes['dis_text'].tolist()
                curr_stay_dict['text_time_to_end'] = curr_stay_notes['icu_time_delta'].values
                #curr_stay_dict['text_embeddings'] = [emb[0][0] for emb in curr_stay_notes['biobert_embeddings']]
                curr_stay_dict['rad_embeddings'] = [
                    np.mean([chunk[0] for chunk in emb], axis=0) for emb in curr_stay_notes['biobert_embeddings']
                ]
                print("shape of rad embeddings: ", curr_stay_notes['biobert_embeddings'].shape)
                curr_stay_dict['note_embeddings'] = [
                    emb[0] for emb in curr_stay_notes['longformer_embeddings']
                ]
                print("shape of note embeddings: ", curr_stay_dict['note_embeddings'][0].shape)
                # Concatenate embeddings for each note
                curr_stay_dict['text_embeddings'] = [np.concatenate((rad_emb, note_emb)) for rad_emb, note_emb in zip(curr_stay_dict['rad_embeddings'], curr_stay_dict['note_embeddings'])]
                print("Length of embeddings: ", len(curr_stay_dict['text_embeddings'][0]))
                curr_stay_dict['text_missing'] = 0
            
        if include_cxr:
            curr_stay_cxr = cxr_df[cxr_df['stay_id'] == curr_stay].copy()
            
            if len(curr_stay_cxr) == 0:
                curr_stay_dict['cxr_feats'] = []
                curr_stay_dict['cxr_time'] = []
                curr_stay_dict['cxr_missing'] = 1
            else:
                curr_stay_dict['cxr_feats'] = curr_stay_cxr['densefeatures'].tolist()
                curr_stay_dict['cxr_time'] = curr_stay_cxr['icu_time_delta'].values
                curr_stay_dict['cxr_missing'] = 0
                #print(f"Number of CXR features: {len(curr_stay_dict['cxr_feats'])}")

        if include_ecg:
            curr_stay_ecg = ecg_df[ecg_df['stay_id'] == curr_stay].copy()
            if len(curr_stay_ecg) == 0:
                curr_stay_dict['ecg_feats'] = []
                curr_stay_dict['ecg_time'] = []
                curr_stay_dict['ecg_missing'] = 1
            else:
                curr_stay_dict['ecg_feats'] = curr_stay_ecg['embeddings'].tolist()
                curr_stay_dict['ecg_time'] = curr_stay_ecg['icu_time_delta'].values
                curr_stay_dict['ecg_missing'] = 0

        curr_stay_dict['label'] = admissions_df[admissions_df['hadm_id'] == curr_stay_dict['hadm_id']]['died'].iloc[0]

        stays_list.append(curr_stay_dict)

    return stays_list



In [35]:
train_stays_list = get_stay_list(train_stays)
val_stays_list = get_stay_list(val_stays)
test_stays_list = get_stay_list(test_stays)

  0%|          | 0/24591 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
# Example to check the first patient stay
first_stay = train_stays[18]
first_stay_data = get_stay_list([first_stay])


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.59it/s]


In [ ]:
# Save the data
import pickle

base_name = "ihm"
if restrict_48_hours:
    base_name += "-48"
else:
    base_name += "-all"

if include_cxr:
    if include_notes:
        base_name += "-cxr-notes"
    else:
        base_name += "-cxr"

if include_ecg:
    base_name += "-ecg"

if include_missing:
    base_name += "-missingInd-two"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving train stays to {f_path}")
    pickle.dump(train_stays_list, f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving val stays to {f_path}")
    pickle.dump(val_stays_list, f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving test stays to {f_path}")
    pickle.dump(test_stays_list, f)


Saving train stays to /data/wang/junh/datasets/multimodal/preprocessing/train_ihm-48-cxr-notes-ecg-missingInd-dis_stays.pkl
Saving val stays to /data/wang/junh/datasets/multimodal/preprocessing/val_ihm-48-cxr-notes-ecg-missingInd-dis_stays.pkl
Saving test stays to /data/wang/junh/datasets/multimodal/preprocessing/test_ihm-48-cxr-notes-ecg-missingInd-dis_stays.pkl


In [2]:
# Save the data
import pickle
file_path = '/data/wang/junh/datasets/multimodal/preprocessing/train_ihm-48-cxr-notes-ecg_stays.pkl'
with open(file_path, 'rb') as f:
    train_stays_list = pickle.load(f)
print(len(train_stays_list))

3624


In [3]:
print(train_stays_list[0].keys())

dict_keys(['name', 'hadm_id', 'stay_id', 'ts_tt', 'irg_ts', 'irg_ts_mask', 'reg_ts', 'text_data', 'text_time_to_end', 'text_embeddings', 'text_missing', 'cxr_feats', 'cxr_time', 'cxr_missing', 'ecg_feats', 'ecg_time', 'ecg_missing', 'label'])


In [13]:
print(len(train_stays_list[1]['reg_ts']))

49
